<a href="https://colab.research.google.com/github/jfish32/Crowdfunding_ETL/blob/main/Copy_of_project4_group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4b1f164844a8bdce20f4e10b9f54a392996da166657d463a765949e082dcc803
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from pyspark.sql import SparkSession
from pathlib import Path

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving realtor-data.zip.csv to realtor-data.zip.csv


In [ ]:
spark = SparkSession.builder \
    .appName("Read CSV with PySpark") \
    .getOrCreate()

In [ ]:
csv_file_path = "realtor-data.zip.csv"

In [ ]:
df = spark.read.csv(str(csv_file_path), header=True, inferSchema=True)
df.show()

+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|prev_sold_date|
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|     Adjuntas|Puerto Rico|     601|     920.0|          NULL|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|     Adjuntas|Puerto Rico|     601|    1527.0|          NULL|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|   Juana Diaz|Puerto Rico|     795|     748.0|          NULL|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|        Ponce|Puerto Rico|     731|    1800.0|          NULL|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|     Mayaguez|Puerto Rico|     680|      NULL|          NULL|
|   103378.0|for_sale|179000.0| 

In [ ]:
df = df.drop("prev_sold_date")
df.show()

+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|     Adjuntas|Puerto Rico|     601|     920.0|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|     Adjuntas|Puerto Rico|     601|    1527.0|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|   Juana Diaz|Puerto Rico|     795|     748.0|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|        Ponce|Puerto Rico|     731|    1800.0|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|     Mayaguez|Puerto Rico|     680|      NULL|
|   103378.0|for_sale|179000.0|  4|   3|    0.46|1850806.0|San Sebastian|Puerto Rico|     612|    2520.0|
|     1205.0|for_sale| 50000.0|  3|   1|     0

In [ ]:
df.count()

2226382

In [ ]:
df_clean = df.dropna()
df_clean.count()

1354105

In [ ]:
df_clean = df_clean.dropDuplicates()
df_clean.count()

1353936

In [ ]:
df_clean.createOrReplaceTempView("data")

In [ ]:
michigan_homes = spark.sql("""
  SELECT *
  FROM data
  WHERE state = 'Michigan'
  """)
michigan_homes.show()

+-----------+--------+--------+---+----+--------+---------+--------------------+--------+--------+----------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|                city|   state|zip_code|house_size|
+-----------+--------+--------+---+----+--------+---------+--------------------+--------+--------+----------+
|    63967.0|for_sale|  9900.0|  2|   1|    0.13| 798491.0|             Inkster|Michigan|   48141|     808.0|
|    98474.0|for_sale|483500.0|  4|   4|     5.0|1538869.0|         Ottawa Lake|Michigan|   49267|    3565.0|
|    75202.0|for_sale|125000.0|  3|   2|    0.14|  81756.0|            Westland|Michigan|   48186|    1135.0|
|    13061.0|for_sale|109900.0|  3|   1|    0.09| 314326.0|             Detroit|Michigan|   48223|    1228.0|
|    35524.0|for_sale|169900.0|  3|   3|    0.41| 545314.0|               Niles|Michigan|   49120|    2100.0|
|    79460.0|for_sale|339900.0|  3|   3|    0.48|1093874.0|            Bridgman|Michigan|   49106|    2196.0|
|    56189

In [ ]:
michigan_homes.count()

22076

In [ ]:
michigan_homes = michigan_homes.drop("state")
michigan_homes = michigan_homes.drop("city")
michigan_homes = michigan_homes.drop("status")
michigan_homes = michigan_homes.dropna()

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

'price'

DataFrame[brokered_by: double, status: string, price: double, bed: int, bath: int, acre_lot: double, street: double, city: string, state: string, zip_code: int, house_size: double, features: vector, features1: vector, features2: vector, features3: vector]

Py4JJavaError: An error occurred while calling o1092.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 32.0 failed 1 times, most recent failure: Lost task 1.0 in stage 32.0 (TID 36) (2412478320a5 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3939/0x000000084159d840`: (struct<brokered_by:double,bed_double_VectorAssembler_5a74b554cf3d:double,bath_double_VectorAssembler_5a74b554cf3d:double,acre_lot:double,street:double,zip_code_double_VectorAssembler_5a74b554cf3d:double,house_size:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedCla